In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import time

from src.Game import Game

In [2]:
print(pd.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2.1.4
Num GPUs Available:  0


In [3]:
tf.keras.backend.clear_session()

In [4]:
bid_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(24 + 1 + 2 + 15,), name="input"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden1"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden2"),
    tf.keras.layers.Dense(11, name="output"),
], name="bid_model")
bid_model.compile(optimizer='adam', loss='mse')

bid_model.summary()

Model: "bid_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 64)                2752      
                                                                 
 hidden2 (Dense)             (None, 64)                4160      
                                                                 
 output (Dense)              (None, 11)                715       
                                                                 
Total params: 7627 (29.79 KB)
Trainable params: 7627 (29.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
play_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(4 + 24 + (6 * 4 * 24) + 16 + 20 + 14 + 30 + 30 + 6 + 2,), name="input"),  # 722
    tf.keras.layers.Dense(64, activation='relu', name="hidden1"),
    tf.keras.layers.Dense(64, activation='relu', name="hidden2"),
    tf.keras.layers.Dense(24, name="output"),
], name="play_model")
play_model.compile(optimizer='adam', loss='mse')

play_model.summary()

Model: "play_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 64)                46272     
                                                                 
 hidden2 (Dense)             (None, 64)                4160      
                                                                 
 output (Dense)              (None, 24)                1560      
                                                                 
Total params: 51992 (203.09 KB)
Trainable params: 51992 (203.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
class BatchLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.batch_logs = []
        self.total_batches_seen = 0
        self.log_df = None

    def on_train_batch_end(self, batch, logs=None):
        self.total_batches_seen += 1
        self.batch_logs.append({'Iteration': self.total_batches_seen, 'Loss': logs['loss'], 'Accuracy': logs['accuracy']})

    def on_epoch_end(self, epoch, logs=None):
        # Create the log DataFrame at the end of each epoch
        self.log_df = pd.DataFrame(self.batch_logs)
logger = BatchLogger()

In [7]:
# Define training parameters
epochs = 1
epsilon = 1.0
epsilon_decay = 0.995
epsilon_min = 0.1
episodes = 1  # 2_000
batch_size = 2 ** 4 #  8192  # 32
gamma = 1  # 0.95

In [8]:
game = Game(bid_model, play_model)

# Play one game with default models
game.play_game()

# Training loop
start_time = time.time()  # Get the current time
for e in range(episodes):
    if e % 100 == 0:
        print(f"Episode {e} | epsilon {epsilon}")
    
    
    

AttributeError: 'str' object has no attribute 'shape'